
# Construção da Tabela Fato – `fato_dex_transacoes`

Este notebook documenta o processo de criação da **tabela fato** `fato_dex_transacoes`, responsável por consolidar os dados transacionais normalizados da camada Bronze em um modelo dimensional analítico. A tabela integra múltiplas dimensões (tempo, token, exchange, rede, contrato) com os fatos quantitativos das transações descentralizadas (DEX).

---

## Objetivo

A `fato_dex_transacoes` tem como propósito armazenar os eventos atômicos de troca de tokens em DEXs, com granularidade no nível de transação. Cada linha representa uma operação registrada na blockchain, enriquecida com chaves substitutas e métricas financeiras, permitindo análises agregadas com flexibilidade e performance.

---

## Etapas do Processo de Criação

### 1. Carregamento da Bronze e das Dimensões

Foram carregadas as seguintes tabelas:
- `dex_trades_bronze` (camada transacional normalizada)
- `dim_calendario`  
- `dim_token` (utilizada duas vezes: para token base e quote)  
- `dim_exchange`  
- `dim_rede`  
- `dim_tipo_contrato`  

### 2. Aplicação de Aliases e Broadcast

Para evitar ambiguidades em joins repetidos (ex: dois joins com `dim_token`), foram atribuídos aliases distintos a cada dimensão. As tabelas pequenas foram otimizadas com o uso de `broadcast()` para acelerar os joins sem causar shuffle.

### 3. Realização dos Joins

Foram executados joins do tipo `left` entre a bronze e cada dimensão, utilizando chaves naturais (ex: `moeda`, `nome_token`, `endereco_token`) para identificar corretamente os `id_*` de cada dimensão. As joins respeitaram o seguinte mapeamento:

- `data_transacao` → `dim_calendario`
- `moeda_base`, `nome_base`, `endereco_base` → `dim_token_base`
- `moeda_quote`, `nome_quote`, `endereco_quote` → `dim_token_quote`
- `exchange_nome`, `exchange_nome_completo`, `protocolo` → `dim_exchange`
- `rede` → `dim_rede`
- `tipo_contrato`, `simbolo_token_contrato`, `nome_token_contrato` → `dim_tipo_contrato`

### 4. Seleção das Colunas Finais

Foram selecionadas:
- As **chaves substitutas** de cada dimensão (`fk_*`)
- As **métricas transacionais**: `quantia_trocada_usd`, `quantia_compra`, `preco_cotacao`, `num_trades`, etc.
- As **medidas operacionais de rede**: `gas`, `preco_gas`, `valor_gas`
- Metadados como `data_ingestao` e `fonte`

A granularidade foi preservada no nível de transação única, sem agregações, para máxima flexibilidade analítica.

### 5. Escrita no Delta Lake

A tabela foi gravada com o comando:

```python
.saveAsTable("workspace.ethereum.fato_dex_transacoes")


In [0]:
# Carregando todas as dimensões
from pyspark.sql.functions import to_date, col, broadcast

dim_calendario_ = broadcast(spark.table("workspace.ethereum.dim_calendario")).alias("dim_calendario")
dim_token_base = broadcast(spark.table("workspace.ethereum.dim_token")).alias("dim_token_base")
dim_token_quote = broadcast(spark.table("workspace.ethereum.dim_token")).alias("dim_token_quote")
dim_exchange_ = broadcast(spark.table("workspace.ethereum.dim_exchange")).alias("dim_exchange")
dim_rede_ = broadcast(spark.table("workspace.ethereum.dim_rede")).alias("dim_rede")
dim_tipo_contrato_ = broadcast(spark.table("workspace.ethereum.dim_tipo_contrato")).alias("dim_tipo_contrato")

#carregando a bronze
df_bronze = spark.table("workspace.ethereum.dex_trades_bronze")
df_bronze_ = df_bronze.alias("bronze")

In [0]:
from pyspark.sql.functions import to_date, col

# Criação da fato
fato = df_bronze_ \
    .join(dim_calendario_, on=to_date(col("bronze.data_transacao")) == col("dim_calendario.data"), how="left") \
    .join(dim_token_base, on=[
        col("bronze.moeda_base") == col("dim_token_base.moeda"),
        col("bronze.nome_base") == col("dim_token_base.nome_token"),
        col("bronze.endereco_base") == col("dim_token_base.endereco_token")
    ], how="left") \
    .join(dim_token_quote, on=[
        col("bronze.moeda_quote") == col("dim_token_quote.moeda"),
        col("bronze.nome_quote") == col("dim_token_quote.nome_token"),
        col("bronze.endereco_quote") == col("dim_token_quote.endereco_token")
    ], how="left") \
    .join(dim_exchange_, on=[
        col("bronze.exchange_nome") == col("dim_exchange.nome_exchange"),
        col("bronze.exchange_nome_completo") == col("dim_exchange.nome_completo"),
        col("bronze.protocolo") == col("dim_exchange.protocolo")
    ], how="left") \
    .join(dim_rede_, col("bronze.rede") == col("dim_rede.rede"), how="left") \
    .join(dim_tipo_contrato_, on=[
        col("bronze.tipo_contrato") == col("dim_tipo_contrato.tipo_contrato"),
        col("bronze.simbolo_token_contrato") == col("dim_tipo_contrato.simbolo_token"),
        col("bronze.nome_token_contrato") == col("dim_tipo_contrato.nome_token")
    ], how="left")

In [0]:
fato_final = fato.select(
    col("dim_calendario.id_data").alias("fk_data"),
    col("dim_token_base.id_token").alias("fk_token_base"),
    col("dim_token_quote.id_token").alias("fk_token_quote"),
    col("dim_exchange.id_exchange").alias("fk_exchange"),
    col("dim_rede.id_rede").alias("fk_rede"),
    col("dim_tipo_contrato.id_tipo_contrato").alias("fk_tipo_contrato"),

    col("bronze.quantia_trocada_usd"),
    col("bronze.quantia_compra"),
    col("bronze.quantia_venda"),
    col("bronze.quantia_base"),
    col("bronze.preco_cotacao"),
    col("bronze.preco_medio"),
    col("bronze.num_trades"),

    col("bronze.direcao"),
    col("bronze.gas"),
    col("bronze.preco_gas"),
    col("bronze.valor_gas"),

    col("bronze.data_ingestao"),
    col("bronze.fonte")
)

# Visto que as minhas análises serão feitas com base em análises de séries temporais, irei particionar a fato por id_data para melhorar a performance
fato_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("fk_data") \
    .saveAsTable("workspace.ethereum.fato_dex_trades")